In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd

In [2]:
client = MongoClient("mongodb://localhost:27017/")

In [3]:
database_names = client.list_database_names()
print(database_names)

['Climate_Fires', 'admin', 'config', 'local']


In [4]:
db = client['Climate_Fires']

In [5]:
collection_names = db.list_collection_names()
print(collection_names)

['Fire_Point', 'Clean_Climate_Data', 'Fire_Area']


In [6]:
Fire_Point = db['Fire_Point']
Clean_Climate_Data = db['Clean_Climate_Data']
Fire_Area = db['Fire_Area']

In [7]:
years = list(range(1976, 2023))
total_fires_may_to_september = 0

# Count the number of matching fires for each year
for year in years:
    # Query fires started from May to September for the current year
    query = {
        "FIRE_START_DATE": {
            "$gte": f"{year}/05/01",
            "$lte": f"{year}/09/30"
        }
    }
    
    num_fires_may_to_september = Fire_Area.count_documents(query)
    total_fires_may_to_september += num_fires_may_to_september
    
    print(f"Number of fires started from May to September in {year}: {num_fires_may_to_september}")

# Print the total sum
print("\nTotal number of fires started from May to September from 1976 to 2022:", total_fires_may_to_september)


Number of fires started from May to September in 1976: 103
Number of fires started from May to September in 1977: 44
Number of fires started from May to September in 1978: 4
Number of fires started from May to September in 1979: 18
Number of fires started from May to September in 1980: 57
Number of fires started from May to September in 1981: 50
Number of fires started from May to September in 1982: 3
Number of fires started from May to September in 1983: 77
Number of fires started from May to September in 1984: 19
Number of fires started from May to September in 1985: 0
Number of fires started from May to September in 1986: 21
Number of fires started from May to September in 1987: 32
Number of fires started from May to September in 1988: 80
Number of fires started from May to September in 1989: 128
Number of fires started from May to September in 1990: 46
Number of fires started from May to September in 1991: 52
Number of fires started from May to September in 1992: 26
Number of fires

In [23]:
# Define the possible FIRE_GENERAL_CAUSE_CODE values
cause_codes = ["IDF", "IDO", "INC", "LTG", "MIS", "REC", "RES", "RWY", "UNK"]

# Initialize a dictionary to count the occurrences of each cause code
cause_code_counts = {code: 0 for code in cause_codes}

# Query documents with FIRE_GENERAL_CAUSE_CODE and specific years, then aggregate counts
pipeline = [
    {
        "$match": {
            "FIRE_GENERAL_CAUSE_CODE": {"$in": cause_codes},
            "FIRE_YEAR": {"$gte": 1976, "$lte": 2023}
        }
    },
    {
        "$group": {
            "_id": "$FIRE_GENERAL_CAUSE_CODE",
            "count": {"$sum": 1}
        }
    }
]

# Execute the aggregation and update cause_code_counts dictionary
result = list(Fire_Area.aggregate(pipeline))
for item in result:
    cause_code_counts[item['_id']] = item['count']

# Print the cause code counts
for code, count in cause_code_counts.items():
    print(f"{code}: {count}")


IDF: 45
IDO: 16
INC: 16
LTG: 2005
MIS: 89
REC: 61
RES: 35
RWY: 46
UNK: 31


In [8]:
query_total = {
    "FIRE_START_DATE": {
        "$gte": "1976/01/01",
        "$lte": "2023/12/31"
    }
}
total_num_fires = Fire_Area.count_documents(query_total)

# Print the total number of fires
print("Total number of fires from 1976 to 2023:", total_num_fires)

Total number of fires from 1976 to 2023: 2345


In [9]:
print("Percentage of Fires started from May to September out of all total fires is 96%")

Percentage of Fires started from May to September out of all total fires is 96%


In [10]:
# Find the largest fire final size
largest_fire_document = Fire_Area.find_one(sort=[('FIRE_FINAL_SIZE', -1)])

if largest_fire_document:
    largest_fire_start_date = largest_fire_document['FIRE_START_DATE']
    largest_fire_final_size = largest_fire_document['FIRE_FINAL_SIZE']

    print("Largest FIRE_FINAL_SIZE:", largest_fire_final_size)
    print("Fire start date for the largest fire:", largest_fire_start_date)
else:
    print("No data found for the largest fire.")

Largest FIRE_FINAL_SIZE: 191811
Fire start date for the largest fire: 2021/06/05 00:00:00+00


In [11]:
query = {
    "LOCAL_DATE": {"$gte": "1976-05", "$lte": "2023-09"}
}

matching_documents = Clean_Climate_Data.find(query)

total_temperature = 0
total_records = 0

# Calculate the total temperature and count of records
for document in matching_documents:
    total_temperature += document['MEAN_TEMPERATURE']
    total_records += 1

# Calculate the average temperature
if total_records > 0:
    average_temperature = total_temperature / total_records
    print("Average temperature from May to September (1976 to 2023):", average_temperature)
else:
    print("No data found for the specified criteria.")


Average temperature from May to September (1976 to 2023): 4.939358994756285


In [13]:
query = {
    "LOCAL_YEAR": {"$gte": 1976, "$lte": 2023}
}

matching_documents = Clean_Climate_Data.find(query)

highest_temperature = float('-inf')  

for document in matching_documents:
    mean_temperature = document['MEAN_TEMPERATURE']
    if mean_temperature > highest_temperature:
        highest_temperature = mean_temperature

if highest_temperature != float('-inf'):
    print("Highest temperature from 1976 to 2023:", highest_temperature)
else:
    print("No data found for the specified years.")

Highest temperature from 1976 to 2023: 24.83703703703704


In [14]:
query = {
    "LOCAL_YEAR": {"$gte": 1976, "$lte": 2023}
}

matching_documents = Clean_Climate_Data.find(query)

highest_temperature = float('-inf')  # Initialize with negative infinity
highest_temperature_date = None

for document in matching_documents:
    mean_temperature = document['MEAN_TEMPERATURE']
    if mean_temperature > highest_temperature:
        highest_temperature = mean_temperature
        highest_temperature_date = document['LOCAL_DATE']

if highest_temperature_date:
    print("Highest temperature date:", highest_temperature_date)
else:
    print("No data found for the specified years.")

Highest temperature date: 2010-07


In [21]:
# Define the possible FIRE_GENERAL_CAUSE_CODE values
cause_codes = ["IDF", "IDO", "INC", "LTG", "MIS", "REC", "RES", "RWY", "UNK"]

# Initialize a dictionary to count the occurrences of each cause code
cause_code_counts = {code: 0 for code in cause_codes}

# Query documents with FIRE_GENERAL_CAUSE_CODE and aggregate counts
pipeline = [
    {
        "$match": {
            "FIRE_GENERAL_CAUSE_CODE": {"$in": cause_codes}
        }
    },
    {
        "$group": {
            "_id": "$FIRE_GENERAL_CAUSE_CODE",
            "count": {"$sum": 1}
        }
    }
]

# Execute the aggregation and update cause_code_counts dictionary
result = list(Fire_Point.aggregate(pipeline))
for item in result:
    cause_code_counts[item['_id']] = item['count']

# Print the cause code counts
for code, count in cause_code_counts.items():
    print(f"{code}: {count}")


IDF: 1215
IDO: 816
INC: 1340
LTG: 30320
MIS: 6550
REC: 11332
RES: 6372
RWY: 4498
UNK: 1262


In [18]:
sample_document = Fire_Point.find_one()

# Print the unique field names in the sample document
unique_field_names = sample_document.keys()
print("Unique Field Names:", unique_field_names)

Unique Field Names: dict_keys(['_id', 'X', 'Y', 'OGF_ID', 'FIRE_DISTURBANCE_AREA_IDENT', 'FIRE_TYPE_CODE', 'FIRE_YEAR', 'FIRE_GENERAL_CAUSE_CODE', 'FIRE_RESPONSE_OBJ_CODE', 'FIRE_START_DATE', 'FIRE_OUT_DATE', 'FIRE_FINAL_SIZE', 'BUSINESS_EFFECTIVE_DATE', 'EFFECTIVE_DATETIME', 'SYSTEM_DATETIME', 'OBJECTID'])
